In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [ ]:
# grouped = ratings[ratings["interaction"]].sort_values("timestamp").groupby(
#     ["day", "userId"]
# ).agg({"movieId": "collect"})
# grouped = grouped.reset_index()
# grouped.head()

In [ ]:
# External dependencies
import os
import cudf  # cuDF is an implementation of Pandas-like Dataframe on GPU

import nvtabular as nvt

In [ ]:
INPUT_DATA_DIR = os.environ.get(
    "INPUT_DATA_DIR", os.path.expanduser("~/nvt-examples/end-to-end-poc/data/")
)

In [ ]:
train_ratings = movie_features_ddf.merge(train, on="movieId")

In [ ]:
train_ratings.head()

In [ ]:
train_ratings.to_parquet(os.path.join(INPUT_DATA_DIR, "train_ratings.parquet"))

In [ ]:
# TODO: Sample a tag from the list for each interaction

sampled_tag = nvt.ColumnGroup(["tags_unique"]) >> (lambda col: len(col))

In [ ]:
output = day_number + sampled_tag
(output).graph

In [ ]:
workflow = nvt.Workflow(output)

In [ ]:
train_ratings_ds = nvt.Dataset(train_ratings)

In [ ]:
workflow.fit(train_ratings_ds)
workflow.transform(train_ratings_ds)

In [ ]:
# TODO: Group by user id and day

In [ ]:
# TODO: Count number of positives per user

In [ ]:
dict_dtypes = {}

for col in CATEGORICAL_COLUMNS:
    dict_dtypes[col] = np.int64

for col in LABEL_COLUMNS:
    dict_dtypes[col] = np.float32

In [ ]:
train_dataset = nvt.Dataset([os.path.join(INPUT_DATA_DIR, "train.parquet")], part_size="100MB")
valid_dataset = nvt.Dataset([os.path.join(INPUT_DATA_DIR, "valid.parquet")], part_size="100MB")

In [ ]:
%%time

workflow.fit(train_dataset)

In [ ]:
# Make sure we have a clean output path
if path.exists(os.path.join(INPUT_DATA_DIR, "train")):
    shutil.rmtree(os.path.join(INPUT_DATA_DIR, "train"))
if path.exists(os.path.join(INPUT_DATA_DIR, "valid")):
    shutil.rmtree(os.path.join(INPUT_DATA_DIR, "valid"))

In [ ]:
%time
workflow.transform(train_dataset).to_parquet(
    output_path=os.path.join(INPUT_DATA_DIR, "train"),
    shuffle=nvt.io.Shuffle.PER_PARTITION,
    cats=["userId", "movieId"],
    labels=["rating"],
    dtypes=dict_dtypes,
)

In [ ]:
%time
workflow.transform(valid_dataset).to_parquet(
    output_path=os.path.join(INPUT_DATA_DIR, "valid"),
    shuffle=False,
    cats=["userId", "movieId"],
    labels=["rating"],
    dtypes=dict_dtypes,
)

In [ ]:
workflow.save(os.path.join(INPUT_DATA_DIR, "workflow"))

In [ ]:
import glob

TRAIN_PATHS = sorted(glob.glob(os.path.join(INPUT_DATA_DIR, "train", "*.parquet")))
VALID_PATHS = sorted(glob.glob(os.path.join(INPUT_DATA_DIR, "valid", "*.parquet")))
TRAIN_PATHS, VALID_PATHS

In [ ]:
df = cudf.read_parquet(TRAIN_PATHS[0])
df.head()